# Task 4 Transmembrane Potential Reconstruction

## PLAN

### 1.1 Task Description

Task 4 requires reconstructing the entire transmembrane potential curve for 75 points in the myocardium based on ECG data. This involves using machine learning to predict the activation function curve over time for each of these points.

### 1.2 Approach

1. **Data Utilization**:
   - Continue using the same dataset as in Task 3, which includes simulated ECG and activation maps.
   - This data mimics the biophysical phenomena occurring during the cardiac cycle, captured through transmembrane voltage recordings.

2. **Model Development**:
   - Develop or adapt neural networks that can interpret the spatial and temporal dynamics of the heart's electrical activity.
   - Expands this to model the continuous change in potential across different cardiac phases.

3. **Training and Evaluation**:
   - Train the neural network to learn from the simulated datasets where both the inputs (ECG data) and targets (transmembrane potential curves) are known.
   - Evaluate the model's accuracy in reconstructing these curves, ensuring it captures the detailed fluctuations in potential that correspond to different cardiac events.

## ANALYZE

### 2.1 Prepare


In [ ]:
# Analysis
import numpy as np
import pandas as pd

import glob, re, os
from typing import List

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Data processing
from torch.utils.data import DataLoader, TensorDataset
from sklearn.utils import shuffle
from torch.optim.lr_scheduler import StepLR

# Save model
import pickle

# Ignore warning
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_rows", 100)
plt.style.use('ggplot')

In [ ]:
# Load data
ECGData = np.load('./combine_dataset/ecg_data.npy')
ActTime = np.load('./combine_dataset/active_time.npy')